## Import Packages

In [248]:
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn import metrics
from joblib import dump, load

## Set seed

In [249]:
random_seed = 8022022 # or any of your favorite number 
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

In [250]:
outcome = "was_preterm"
combine_outcome = True

## Read dataset

### read metadata

In [251]:
metadata = pd.read_csv('data/raw_combine/comb_meta_alpha.csv', delimiter=',',index_col = 'specimen').sort_index()

In [252]:
print(metadata.shape)
metadata.head(5)

(2461, 14)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,inv_simpson,bwpd,phylo_entropy,quadratic,unrooted_pd,rooted_pd
specimen,,,,,,,,,,,,,,
A00001-05,A00001,A,False,False,38,33,III,1.00000,1.00000,0.00000,0.00000,0.000000,0.00000,2.53935
A00002-01,A00002,A,False,False,40,38,III,1.96362,1.81277,2.62894,1.31887,0.876314,3.94341,4.14816
A00003-02,A00003,A,False,False,40,30,II,1.00000,1.00000,0.00000,0.00000,0.000000,0.00000,2.62632
A00004-08,A00004,A,False,False,40,27,III,1.00000,1.00000,0.00000,0.00000,0.000000,0.00000,1.83870
A00004-12,A00004,A,False,False,40,29,III,6.94884,4.07385,2.78896,3.13422,1.219900,15.51850,15.58460


### read taxonomy

In [253]:
taxonomy1 = pd.read_csv('data/raw_combine/comb_taxonomy_relabd.family.csv', delimiter=',',index_col = 'specimen')
tax_feature1 = list(pd.read_csv('data/selected_feature/tax_family_preterm.csv', delimiter=',').feature_selected)
taxonomy_selected1 = taxonomy1[tax_feature1].sort_index()
tax_old_name1 = list(taxonomy_selected1.columns)
tax_new_name1 = [item + '_tax1' for item in tax_old_name1]
tax_renamedict1 = dict(zip(tax_old_name1,tax_new_name1))

taxonomy_selected1 = taxonomy_selected1.rename(columns = tax_renamedict1)

In [254]:
taxonomy2 = pd.read_csv('data/raw_combine/comb_taxonomy_relabd.genus.csv', delimiter=',',index_col = 'specimen')
tax_feature2 = list(pd.read_csv('data/selected_feature/tax_genus_preterm.csv', delimiter=',').feature_selected)
taxonomy_selected2 = taxonomy2[tax_feature2].sort_index()
tax_old_name2 = list(taxonomy_selected2.columns)
tax_new_name2 = [item + '_tax2' for item in tax_old_name2]
tax_renamedict2 = dict(zip(tax_old_name2,tax_new_name2))

taxonomy_selected2 = taxonomy_selected2.rename(columns = tax_renamedict2)

In [255]:
taxonomy3 = pd.read_csv('data/raw_combine/comb_taxonomy_relabd.species.csv', delimiter=',',index_col = 'specimen')
tax_feature3 = list(pd.read_csv('data/selected_feature/tax_species_preterm.csv', delimiter=',').feature_selected)
taxonomy_selected3 = taxonomy3[tax_feature3].sort_index()
tax_old_name3 = list(taxonomy_selected3.columns)
tax_new_name3 = [item + '_tax3' for item in tax_old_name3]
tax_renamedict3 = dict(zip(tax_old_name3,tax_new_name3))

taxonomy_selected3 = taxonomy_selected3.rename(columns = tax_renamedict3)

In [256]:
taxonomy_selected = pd.concat([taxonomy_selected1,taxonomy_selected2,taxonomy_selected3], axis = 1).copy()

In [257]:
taxonomy_selected

,Prevotellaceae_tax1,Corynebacteriaceae_tax1,unclassified.Clostridiales_tax1,Staphylococcaceae_tax1,Bifidobacteriaceae_tax1,Lachnospiraceae_tax1,Fusobacteriaceae_tax1,unclassified.Corynebacteriales_tax1,Ruminococcaceae_tax1,Mycoplasmataceae_tax1,...,Prevotella_tax3,Lactobacillus.crispatus_tax3,Prevotella.melaninogenica_tax3,Streptococcus.mitis_tax3,Lawsonella.clevelandensis_tax3,Bacteroidales_tax3,Lachnospiraceae_tax3,Fenollaria.massiliensis.timonensis_tax3,Prevotellaceae_tax3,Anaerococcus.prevotii.tetradius_tax3
specimen,,,,,,,,,,,,,,,,,,,,,
A00001-05,0.113386,0.000000,0.022572,0.003675,0.001050,0.001050,0.00105,0.0,0.0,0.003150,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.022572,0.090289,0.0
A00002-01,0.011649,0.000000,0.000000,0.000000,0.000000,0.044145,0.00000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000613,0.0
A00003-02,0.011874,0.000000,0.000000,0.000000,0.001079,0.000000,0.00000,0.0,0.0,0.000000,...,0.000216,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000216,0.0
A00004-08,0.031860,0.000000,0.000514,0.000514,0.000000,0.020041,0.00000,0.0,0.0,0.002569,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000514,0.027235,0.0
A00004-12,0.143407,0.000366,0.002564,0.000000,0.000000,0.008425,0.00000,0.0,0.0,0.003846,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.142857,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J00111-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.004992,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
J00112-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.842958,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
J00113-01,0.000631,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.004733,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


### read phylotype

In [258]:
phylotype1 = pd.read_csv('data/raw_combine/comb_phylotype_relabd.1e_1.csv', delimiter=',',index_col = 'specimen')
phylo_feature1 = list(pd.read_csv('data/selected_feature/phylo_.1_preterm.csv', delimiter=',').feature_selected)
phylotype_selected1 = phylotype1[phylo_feature1].sort_index()
phylo_old_name1 = list(phylotype_selected1.columns)
phylo_new_name1 = [item + '_phylo1' for item in phylo_old_name1]
phylo_renamedict1 = dict(zip(phylo_old_name1,phylo_new_name1))

phylotype_selected1 = phylotype_selected1.rename(columns = phylo_renamedict1)

In [259]:
phylotype2 = pd.read_csv('data/raw_combine/comb_phylotype_relabd.5e_1.csv', delimiter=',',index_col = 'specimen')
phylo_feature2 = list(pd.read_csv('data/selected_feature/phylo_.5_preterm.csv', delimiter=',').feature_selected)
phylotype_selected2 = phylotype2[phylo_feature2].sort_index()
phylo_old_name2 = list(phylotype_selected2.columns)
phylo_new_name2 = [item + '_phylo2' for item in phylo_old_name2]
phylo_renamedict2 = dict(zip(phylo_old_name2,phylo_new_name2))

phylotype_selected2 = phylotype_selected2.rename(columns = phylo_renamedict2)

In [260]:
phylotype3 = pd.read_csv('data/raw_combine/comb_phylotype_relabd.1e0.csv', delimiter=',',index_col = 'specimen')
phylo_feature3 = list(pd.read_csv('data/selected_feature/phylo_1_preterm.csv', delimiter=',').feature_selected)
phylotype_selected3 = phylotype3[phylo_feature3].sort_index()
phylo_old_name3 = list(phylotype_selected3.columns)
phylo_new_name3 = [item + '_phylo3' for item in phylo_old_name3]
phylo_renamedict3 = dict(zip(phylo_old_name3,phylo_new_name3))

phylotype_selected3 = phylotype_selected3.rename(columns = phylo_renamedict3)

In [261]:
phylotype_selected = pd.concat([phylotype_selected1,phylotype_selected2,phylotype_selected3], axis = 1).copy()

In [262]:
phylotype_selected

,pt__00004_phylo1,pt__00090_phylo1,pt__00056_phylo1,pt__00042_phylo1,pt__00079_phylo1,pt__00024_phylo1,pt__00071_phylo1,pt__00092_phylo1,pt__00096_phylo1,pt__00007_phylo1,...,pt__00006_phylo3,pt__00030_phylo3,pt__00010_phylo3,pt__00027_phylo3,pt__00013_phylo3,pt__00035_phylo3,pt__00056_phylo3,pt__00007_phylo3,pt__00001_phylo3,pt__00009_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00001-05,0.000000,0.0,0.0,0.022572,0.0,0.0,0.000000,0.0,0.0,0.003675,...,0.000000,0.00105,0.0,0.0,0.001575,0.0,0.0,0.106037,0.797900,0.009449
A00002-01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000613,0.0,0.0,0.000000,...,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000613,0.805641,0.001839
A00003-02,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00000,0.0,0.0,0.000648,0.0,0.0,0.000216,0.963299,0.001079
A00004-08,0.000000,0.0,0.0,0.000514,0.0,0.0,0.000000,0.0,0.0,0.000514,...,0.000000,0.00000,0.0,0.0,0.000514,0.0,0.0,0.027235,0.927544,0.000000
A00004-12,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000183,0.0,0.0,0.000000,...,0.000366,0.00000,0.0,0.0,0.001465,0.0,0.0,0.142857,0.806593,0.000916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J00111-01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000
J00112-01,0.141649,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.999644,0.000000
J00113-01,0.001736,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.483433,0.000000


### combine data

In [263]:
mydata = pd.concat([metadata,taxonomy_selected,phylotype_selected], axis = 1).copy()

In [264]:
print(mydata.shape)
mydata.head(5)

(2461, 104)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,inv_simpson,bwpd,...,pt__00006_phylo3,pt__00030_phylo3,pt__00010_phylo3,pt__00027_phylo3,pt__00013_phylo3,pt__00035_phylo3,pt__00056_phylo3,pt__00007_phylo3,pt__00001_phylo3,pt__00009_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00001-05,A00001,A,False,False,38,33,III,1.00000,1.00000,0.00000,...,0.000000,0.00105,0.0,0.0,0.001575,0.0,0.0,0.106037,0.797900,0.009449
A00002-01,A00002,A,False,False,40,38,III,1.96362,1.81277,2.62894,...,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000613,0.805641,0.001839
A00003-02,A00003,A,False,False,40,30,II,1.00000,1.00000,0.00000,...,0.000000,0.00000,0.0,0.0,0.000648,0.0,0.0,0.000216,0.963299,0.001079
A00004-08,A00004,A,False,False,40,27,III,1.00000,1.00000,0.00000,...,0.000000,0.00000,0.0,0.0,0.000514,0.0,0.0,0.027235,0.927544,0.000000
A00004-12,A00004,A,False,False,40,29,III,6.94884,4.07385,2.78896,...,0.000366,0.00000,0.0,0.0,0.001465,0.0,0.0,0.142857,0.806593,0.000916


### change data type

In [265]:
mydata["project"] = mydata["project"].astype('category')
mydata["CST"] = mydata["CST"].astype('category')
mydata['was_preterm'] = mydata['was_preterm'].astype('int64')
mydata['was_early_preterm'] = mydata['was_early_preterm'].astype('int64')

mydata.dtypes

participant_id         object
project              category
was_preterm             int64
was_early_preterm       int64
delivery_wk             int64
                       ...   
pt__00035_phylo3      float64
pt__00056_phylo3      float64
pt__00007_phylo3      float64
pt__00001_phylo3      float64
pt__00009_phylo3      float64
Length: 104, dtype: object

### subset data by outcome

In [266]:
# only keep the rows with collect_wk < 32 for Preterm task
mydata_preterm_t = mydata.loc[mydata['collect_wk']<=32,].copy().sort_index(axis = 0)

mydata_preterm = mydata_preterm_t.drop(columns = ['inv_simpson','bwpd','phylo_entropy','quadratic','unrooted_pd']) 
# mydata_preterm = mydata_preterm_t.loc[mydata_preterm_t['project']!= 'I',].copy().sort_index(axis = 0)
print(mydata_preterm.shape)

mydata_preterm.head(5)

(2077, 99)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,rooted_pd,Prevotellaceae_tax1,...,pt__00006_phylo3,pt__00030_phylo3,pt__00010_phylo3,pt__00027_phylo3,pt__00013_phylo3,pt__00035_phylo3,pt__00056_phylo3,pt__00007_phylo3,pt__00001_phylo3,pt__00009_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00003-02,A00003,A,0,0,40,30,II,1.00000,2.62632,0.011874,...,0.000000,0.0,0.0,0.000000,0.000648,0.0,0.000000,0.000216,0.963299,0.001079
A00004-08,A00004,A,0,0,40,27,III,1.00000,1.83870,0.031860,...,0.000000,0.0,0.0,0.000000,0.000514,0.0,0.000000,0.027235,0.927544,0.000000
A00004-12,A00004,A,0,0,40,29,III,6.94884,15.58460,0.143407,...,0.000366,0.0,0.0,0.000000,0.001465,0.0,0.000000,0.142857,0.806593,0.000916
A00004-13,A00004,A,0,0,40,30,III,3.04127,6.58140,0.221710,...,0.000000,0.0,0.0,0.003072,0.000341,0.0,0.000000,0.221540,0.686465,0.000512
A00005-02,A00005,A,0,0,41,26,II,1.85087,3.45792,0.005714,...,0.000000,0.0,0.0,0.001905,0.000952,0.0,0.000476,0.000476,0.755714,0.001905


In [267]:
mydata_preterm_ind = mydata_preterm.groupby('participant_id').first().copy()

mydata_preterm_ind.shape

(1068, 98)

### split train validation id set

In [268]:
test_id = list(mydata_preterm_ind.groupby(outcome).sample(frac = 0.2, random_state = 200).sort_index(axis = 0).index)

# train_val_id = list(mydata_preterm_ind.loc[~mydata_preterm_ind.index.isin(train_id)].groupby('was_preterm').\
#             sample(frac = 0.5, random_state = 200).sort_index(axis = 0).index)

train_val_id = list(np.setdiff1d(list(mydata_preterm_ind.index),test_id))

print(len(test_id))
print(len(train_val_id))


213
855


### obtain train validation dataset

In [269]:
test_data = mydata_preterm[mydata_preterm['participant_id'].isin(test_id)]

print(len(test_id))
print(test_data.shape)
test_data.head(5)

213
(382, 99)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,rooted_pd,Prevotellaceae_tax1,...,pt__00006_phylo3,pt__00030_phylo3,pt__00010_phylo3,pt__00027_phylo3,pt__00013_phylo3,pt__00035_phylo3,pt__00056_phylo3,pt__00007_phylo3,pt__00001_phylo3,pt__00009_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00004-08,A00004,A,0,0,40,27,III,1.00000,1.83870,0.031860,...,0.000000,0.000000,0.0,0.000000,0.000514,0.000000,0.000000,0.027235,0.927544,0.000000
A00004-12,A00004,A,0,0,40,29,III,6.94884,15.58460,0.143407,...,0.000366,0.000000,0.0,0.000000,0.001465,0.000000,0.000000,0.142857,0.806593,0.000916
A00004-13,A00004,A,0,0,40,30,III,3.04127,6.58140,0.221710,...,0.000000,0.000000,0.0,0.003072,0.000341,0.000000,0.000000,0.221540,0.686465,0.000512
A00010-01,A00010,A,0,0,38,27,III,1.00000,1.47978,0.094606,...,0.009037,0.005366,0.0,0.000847,0.000282,0.000000,0.000000,0.036430,0.607738,0.011579
A00010-02,A00010,A,0,0,38,28,III,2.06211,3.78214,0.181584,...,0.006001,0.008745,0.0,0.001372,0.002743,0.000171,0.004115,0.026920,0.531036,0.023834


In [270]:
train_val_data = mydata_preterm[mydata_preterm['participant_id'].isin(train_val_id)]

print(len(train_val_id))
print(train_val_data.shape)
train_val_data.head(5)

855
(1695, 99)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,rooted_pd,Prevotellaceae_tax1,...,pt__00006_phylo3,pt__00030_phylo3,pt__00010_phylo3,pt__00027_phylo3,pt__00013_phylo3,pt__00035_phylo3,pt__00056_phylo3,pt__00007_phylo3,pt__00001_phylo3,pt__00009_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00003-02,A00003,A,0,0,40,30,II,1.00000,2.62632,0.011874,...,0.000000,0.0,0.0,0.000000,0.000648,0.0,0.000000,0.000216,0.963299,0.001079
A00005-02,A00005,A,0,0,41,26,II,1.85087,3.45792,0.005714,...,0.000000,0.0,0.0,0.001905,0.000952,0.0,0.000476,0.000476,0.755714,0.001905
A00005-05,A00005,A,0,0,41,29,II,1.00000,1.73089,0.001582,...,0.000395,0.0,0.0,0.000395,0.000000,0.0,0.000000,0.000000,0.913405,0.000395
A00006-01,A00006,A,0,0,41,31,II,1.00000,1.23463,0.030000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.030000,0.762941,0.084118
A00008-02,A00008,A,1,0,35,17,IV-B,1.00000,1.58820,0.090226,...,0.000000,0.0,0.0,0.000000,0.003759,0.0,0.000000,0.000000,0.156391,0.097744


In [286]:
def tensor_generator(data, outcome = outcome, combine_outcome = combine_outcome):
    
    if combine_outcome:
        X_data = data[data.columns[6:]].copy()
        X_data['CST'] = X_data['CST'].cat.codes
        X_feature = X_data.to_numpy().astype('float32')
        X_group = data['participant_id'].astype('category').cat.codes.values.reshape(-1,1)
        X_cw = data['collect_wk'].values.reshape(-1,1)
        input_X = torch.from_numpy(np.hstack((X_feature,X_group,X_cw)))
        y_data = data.groupby('participant_id').first()
        y = y_data[outcome].values.flatten()
        input_y = torch.from_numpy(np.vstack((y,1-y)).T.astype('float32'))
    else:
        X_data = data[data.columns[5:]].copy()
        X_data['CST'] = X_data['CST'].cat.codes
        X_feature = X_data.to_numpy().astype('float32')
        input_X = torch.from_numpy(X_feature)
        y = data[outcome].values.flatten()
        input_y = torch.from_numpy(np.vstack((y,1-y)).T.astype('float32'))
    
    
    return input_X, input_y

    

In [287]:
test_data[test_data.columns[6:]].columns

Index(['CST', 'shannon', 'rooted_pd', 'Prevotellaceae_tax1',
       'Corynebacteriaceae_tax1', 'unclassified.Clostridiales_tax1',
       'Staphylococcaceae_tax1', 'Bifidobacteriaceae_tax1',
       'Lachnospiraceae_tax1', 'Fusobacteriaceae_tax1',
       'unclassified.Corynebacteriales_tax1', 'Ruminococcaceae_tax1',
       'Mycoplasmataceae_tax1', 'unclassified.Tissierellia_tax1',
       'Veillonellaceae_tax1', 'Lactobacillaceae_tax1', 'Bacteroidales_tax1',
       'Alcaligenaceae_tax1', 'Prevotella_tax2', 'Fenollaria_tax2',
       'Mycoplasma_tax2', 'Ezakiella_tax2', 'Varibaculum_tax2',
       'Lawsonella_tax2', 'Corynebacterium_tax2', 'Staphylococcus_tax2',
       'Fusobacterium_tax2', 'Bifidobacterium_tax2', 'Winkia_tax2',
       'Lachnospiraceae_tax2', 'Bacteroidales_tax2', 'Prevotellaceae_tax2',
       'Lactobacillus_tax2', 'Lactobacillus.jensenii_tax3',
       'Corynebacterium.tuberculostearicum_tax3',
       'Fusobacterium.nucleatum_tax3', 'Anaerococcus.mediterraneensis_tax3',
    

In [329]:
# Define RNN model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1,hidden_dim2,
                 layer3, hidden_dim3,
                 drop_out,combine_outcome):
        

        #inherit from super class
        super(MLP, self).__init__()
        
        #define combine 
        
        self.combine = combine_outcome
        self.layer3 = layer3
        
        #define layers
        if layer3:
            self.fc1 = nn.Linear(input_dim, hidden_dim1)
            self.fc2 = nn.Linear(hidden_dim1,hidden_dim2)
            self.fc3 = nn.Linear(hidden_dim2,hidden_dim3)
            self.fc4 = nn.Linear(hidden_dim3,2)
        else:
            self.fc1 = nn.Linear(input_dim, hidden_dim1)
            self.fc2 = nn.Linear(hidden_dim1,hidden_dim2)
            self.fc3 = nn.Linear(hidden_dim2,2)

        self.dropout = nn.Dropout(drop_out)

        
    def forward(self, x):
        
        if self.combine:
        
            X_feature = x[:,:-2].float()
            X_group = x[:,-2].long()
            X_cw = x[:,-1].long()
            
            
            X_feature = torch.tanh(self.fc1(X_feature))
            X_feature = self.dropout(X_feature)
            X_feature = F.relu(self.fc2(X_feature))
            X_feature = self.dropout(X_feature)
            X_feature = torch.tanh(self.fc3(X_feature))
            
            if self.layer3:
                X_feature = self.dropout(X_feature)
                X_feature = torch.tanh(self.fc4(X_feature))

            X_feature = F.softmax(X_feature, dim = 1)

            M = torch.zeros(X_group.max()+1, len(X_feature))

            for i in range(len(X_feature)):
                M[X_group[i],i] = X_cw[i]

            M = F.normalize(M, p=1, dim=1)
            X_feature = torch.mm(M, X_feature)
            
            
        else: 
            
            X_feature = x
            
            X_feature = torch.tanh(self.fc1(X_feature))
            X_feature = self.dropout(X_feature)
            X_feature = F.relu(self.fc2(X_feature))
            X_feature = self.dropout(X_feature)
            X_feature = torch.tanh(self.fc3(X_feature))
            
            if self.layer3:
                X_feature = self.dropout(X_feature)
                X_feature = torch.tanh(self.fc4(X_feature))

            X_feature = F.softmax(X_feature, dim = 1)
        
        return X_feature
    

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01) 


In [330]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [331]:
def indloader(data, seed, outcome = outcome):
    data_ind = data.groupby('participant_id').first().copy()
    id_1 = list(data_ind.groupby(outcome).sample(frac = 0.5, random_state = seed).sort_index(axis = 0).index)
    id_2 = list(np.setdiff1d(list(data_ind.index),id_1))
    
    return [id_1,id_2]
    

In [332]:
def train_epoch(train_data, val_data, learn_rate, \
                hidden_dim1 = 10,hidden_dim2 = 10,layer3 = False,\
                hidden_dim3 = 0,drop_out = 0.1, \
                device = device, EPOCHS = 500, patience = 20,combine_outcome = combine_outcome,outcome = outcome):
    
    val_X, val_y = tensor_generator(val_data, outcome = outcome, combine_outcome = combine_outcome)
    
    
    #input_dim
    if combine_outcome:
        input_dim = val_X.shape[1]-2
    else:
        input_dim = val_X.shape[1]
    

    model = MLP(input_dim, hidden_dim1,hidden_dim2,layer3, hidden_dim3,
                 drop_out,combine_outcome)

        
    model.to(device)
    
    model.apply(init_weights)
    # loss criterion and optimizer

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learn_rate)
    
    train_loss = []
    val_loss = []
    # train model
    
    model.train()
    
    last_loss = 1000
    trigger_times = 0
    #Start training loop
    
    for epoch in range(1, EPOCHS + 1):
        
        batch_train_losses = []
        ind_loader = indloader(train_data, seed = 980 + epoch, outcome = outcome)
        
        
        for ind in ind_loader:
            
            train_batch = train_data[train_data['participant_id'].isin(ind)]
            
            x, label = tensor_generator(train_batch,outcome = outcome, combine_outcome = combine_outcome)
            
            x, label = x.to(device), label.to(device)
            
            model.zero_grad()
        
            predictions = model(x)
            
            predictions = predictions.to(device)
        
            loss = criterion(predictions, label)
            
            batch_train_losses.append(loss.detach().numpy())
        
            # backpropagation
            loss.backward() 
            # Updates the weights accordingly
            optimizer.step()
        
        train_loss.append(np.mean(batch_train_losses))
        
        with torch.no_grad():
            
            model.eval()
            
            val_X, val_y = val_X.to(device), val_y.to(device)
                
            predictions_val = model(val_X)
                
            predictions_val = predictions_val.to(device)
                
            loss_val = criterion(predictions_val, val_y)
            
            val_loss.append(loss_val.detach().numpy())
            

        current_loss = val_loss[epoch-1]
        
        
        if current_loss >= last_loss:
            trigger_times += 1
            
            if trigger_times >= patience:
                print ('Early stopping at {}'.format(epoch))
                return model,train_loss,val_loss
        else:
            tigger_times = 0
        
        last_loss = current_loss
        
        if epoch%10 == 0:
            print('Epoch: {}/{}.............'.format(epoch, EPOCHS), end = ' ')
            print("Train Loss: {:.4f}".format(train_loss[epoch-1]), end = ' ')
            print("Validation Loss: {:.4f}".format(val_loss[epoch-1]))
            
        
    return model,train_loss,val_loss


smaller batch size (2^)
epoch (early stopping)
residual net

In [333]:
def test_metrics(model, val_data, outcome = outcome, combine_outcome = combine_outcome):
    
    val_X = tensor_generator(val_data, outcome = outcome, combine_outcome = combine_outcome)[0]
    
    if combine_outcome:
        val_y = val_data.groupby('participant_id').first()[outcome].values.flatten()
    else:
        val_y = val_data[outcome].values.flatten()
    
    model.eval()
    

    out = model(val_X)
    predicted_props = out[:,0].detach().numpy()
    predicted_labels = 1*(predicted_props >=0.5)
    
    
    acc = metrics.accuracy_score(val_y, predicted_labels, normalize=False) / float(val_y.size)
    confusion = metrics.confusion_matrix(val_y, predicted_labels)
    

    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    
    specificity = TN / (TN + FP)
    sensitivity = TP / (TP + FN)
    precision = TP/(TP + FP)
    
    auc = metrics.roc_auc_score(val_y, predicted_props)
    
    if combine_outcome:
    
        result_tab = pd.DataFrame(data = {'participant_id': list(val_data.groupby('participant_id').first().index),
                                         'labels':list(val_y),
                                         'predicted_labels':list(predicted_labels),
                                         'predicted_probs':list(predicted_props)})
    else:
        result_tab = pd.DataFrame(data = {'participant_id': list(val_data.index),
                                         'labels':list(val_y),
                                         'predicted_labels':list(predicted_labels),
                                         'predicted_probs':list(predicted_props)})
    

    
    return acc,sensitivity,specificity,auc,precision,confusion,result_tab


In [334]:
import itertools

#(0.05, 16, 32, False, 64, 0.2)

# a = [[0.001,0.05], \
#     [16,32,64], \
#     [16,32,64], \
#     [False,True], \
#     [16,32,64], \
#     [0.05,0.1,0.2]]

a = [[0.05], \
    [16,], \
    [16,32,64], \
    [False,True], \
    [16], \
    [0.05,0.1,0.2]]




parameter_space = list(itertools.product(*a))

len(parameter_space)



324

In [335]:
def parameter_tuning(train_val_data, parameter_space,cv_time =5,seed = 999,outcome = outcome, combine_outcome = combine_outcome):
    
    best_parameter = tuple()
    best_auc = 0
    best_model = None
    
    train_val_data_ind = train_val_data.groupby('participant_id').first().copy()
    
    for param in parameter_space:
        
        auc_cv = []
        
        for i in range(cv_time):
            
            train_id = list(train_val_data_ind.groupby(outcome).sample(frac = 0.75, random_state = seed + i).sort_index(axis = 0).index)
            val_id =  list(np.setdiff1d(list(train_val_data_ind.index),train_id))
            
            train_data = train_val_data[train_val_data['participant_id'].isin(train_id)]
            val_data = train_val_data[train_val_data['participant_id'].isin(val_id)]
        
            model = train_epoch(train_data,val_data,learn_rate = param[0], \
                               hidden_dim1= param[1], hidden_dim2= param[2], \
                               layer3 = param[3], hidden_dim3= param[4],drop_out= param[5], device = device)[0]
        
            auc = test_metrics(model, val_data, outcome = outcome, combine_outcome = combine_outcome)[3]
            
            auc_cv.append(auc)
        
        auc_ave = np.mean(auc_cv)
        
        if auc_ave >= best_auc:
            best_auc = auc_ave
            best_parameter = param
            best_model = model
            
    return best_parameter, best_model,best_auc
        
    

    

In [336]:
best_parameter, best_model,best_auc = parameter_tuning(train_val_data,parameter_space,cv_time =3,seed = 999,outcome = outcome)

Epoch: 10/500............. Train Loss: 0.6963 Validation Loss: 0.6957
Epoch: 20/500............. Train Loss: 0.6742 Validation Loss: 0.6739
Epoch: 30/500............. Train Loss: 0.6475 Validation Loss: 0.6489
Epoch: 40/500............. Train Loss: 0.6399 Validation Loss: 0.6415
Epoch: 50/500............. Train Loss: 0.6385 Validation Loss: 0.6395
Epoch: 60/500............. Train Loss: 0.6372 Validation Loss: 0.6375
Epoch: 70/500............. Train Loss: 0.6350 Validation Loss: 0.6346
Epoch: 80/500............. Train Loss: 0.6314 Validation Loss: 0.6301
Epoch: 90/500............. Train Loss: 0.6271 Validation Loss: 0.6257
Epoch: 100/500............. Train Loss: 0.6236 Validation Loss: 0.6234
Epoch: 110/500............. Train Loss: 0.6212 Validation Loss: 0.6218
Epoch: 120/500............. Train Loss: 0.6190 Validation Loss: 0.6204
Epoch: 130/500............. Train Loss: 0.6169 Validation Loss: 0.6190
Epoch: 140/500............. Train Loss: 0.6147 Validation Loss: 0.6176
Epoch: 150/500.

Epoch: 80/500............. Train Loss: 0.6281 Validation Loss: 0.6304
Epoch: 90/500............. Train Loss: 0.6237 Validation Loss: 0.6268
Epoch: 100/500............. Train Loss: 0.6209 Validation Loss: 0.6251
Epoch: 110/500............. Train Loss: 0.6182 Validation Loss: 0.6240
Epoch: 120/500............. Train Loss: 0.6153 Validation Loss: 0.6229
Epoch: 130/500............. Train Loss: 0.6116 Validation Loss: 0.6232
Epoch: 140/500............. Train Loss: 0.6086 Validation Loss: 0.6233
Epoch: 150/500............. Train Loss: 0.6064 Validation Loss: 0.6210
Epoch: 160/500............. Train Loss: 0.6047 Validation Loss: 0.6193
Epoch: 170/500............. Train Loss: 0.6032 Validation Loss: 0.6178
Epoch: 180/500............. Train Loss: 0.6019 Validation Loss: 0.6164
Epoch: 190/500............. Train Loss: 0.6007 Validation Loss: 0.6148
Epoch: 200/500............. Train Loss: 0.5994 Validation Loss: 0.6137
Epoch: 210/500............. Train Loss: 0.5982 Validation Loss: 0.6125
Epoch: 2

Epoch: 90/500............. Train Loss: 0.6255 Validation Loss: 0.6238
Epoch: 100/500............. Train Loss: 0.6215 Validation Loss: 0.6201
Epoch: 110/500............. Train Loss: 0.6168 Validation Loss: 0.6157
Epoch: 120/500............. Train Loss: 0.6120 Validation Loss: 0.6112
Epoch: 130/500............. Train Loss: 0.6071 Validation Loss: 0.6071
Epoch: 140/500............. Train Loss: 0.6023 Validation Loss: 0.6032
Epoch: 150/500............. Train Loss: 0.5980 Validation Loss: 0.6003
Epoch: 160/500............. Train Loss: 0.5946 Validation Loss: 0.5962
Epoch: 170/500............. Train Loss: 0.5923 Validation Loss: 0.5942
Epoch: 180/500............. Train Loss: 0.5905 Validation Loss: 0.5926
Epoch: 190/500............. Train Loss: 0.5890 Validation Loss: 0.5912
Epoch: 200/500............. Train Loss: 0.5880 Validation Loss: 0.5901
Epoch: 210/500............. Train Loss: 0.5873 Validation Loss: 0.5892
Epoch: 220/500............. Train Loss: 0.5863 Validation Loss: 0.5885
Epoch: 

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_21104\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/500............. Train Loss: 0.6684 Validation Loss: 0.6677
Epoch: 20/500............. Train Loss: 0.6533 Validation Loss: 0.6533
Epoch: 30/500............. Train Loss: 0.6409 Validation Loss: 0.6419
Epoch: 40/500............. Train Loss: 0.6387 Validation Loss: 0.6393
Epoch: 50/500............. Train Loss: 0.6372 Validation Loss: 0.6370
Epoch: 60/500............. Train Loss: 0.6340 Validation Loss: 0.6330
Epoch: 70/500............. Train Loss: 0.6295 Validation Loss: 0.6281
Epoch: 80/500............. Train Loss: 0.6261 Validation Loss: 0.6258
Epoch: 90/500............. Train Loss: 0.6235 Validation Loss: 0.6243
Epoch: 100/500............. Train Loss: 0.6212 Validation Loss: 0.6229
Epoch: 110/500............. Train Loss: 0.6189 Validation Loss: 0.6214
Epoch: 120/500............. Train Loss: 0.6161 Validation Loss: 0.6198
Epoch: 130/500............. Train Loss: 0.6136 Validation Loss: 0.6190
Epoch: 140/500............. Train Loss: 0.6115 Validation Loss: 0.6168
Epoch: 150/500.

...

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_21104\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/500............. Train Loss: 0.6418 Validation Loss: 0.6406
Epoch: 20/500............. Train Loss: 0.6418 Validation Loss: 0.6406
Early stopping at 21


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_21104\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/500............. Train Loss: 0.6418 Validation Loss: 0.6406
Epoch: 20/500............. Train Loss: 0.6418 Validation Loss: 0.6406
Early stopping at 30


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_21104\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


In [337]:
best_auc

0.6667487522983976

In [338]:
best_parameter

(0.05, 16, 32, False, 64, 0.2)

In [325]:
test_acc,test_sensitivity,test_specificity, \
test_auc,test_precision,test_confusion,test_result_tab = \
test_metrics(best_model, test_data, outcome = outcome)


In [326]:
test_result_tab

,participant_id,labels,predicted_labels,predicted_probs
0,A00004,0,0,0.121610
1,A00010,0,0,0.384625
2,A00029,0,1,0.621522
3,A00031,0,0,0.119361
4,B00003,0,0,0.126645
...,...,...,...,...
208,J00093,0,0,0.252475
209,J00100,0,0,0.119866
210,J00102,0,0,0.231561
211,J00105,0,0,0.119923


In [327]:
best_model

MLP(
  (fc1): Linear(in_features=93, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [64]:
dump(best_model,'Submission/test_submission.save')

['Submission/test_submission.save']

In [328]:
test_result_tab.to_csv('test_error_analysis.csv')

In [168]:
test_metrics(best_model, train_val_data, outcome = outcome)[6].to_csv('train_val_error_analysis.csv')

In [ ]:
# def project_cross_val(features,train_ratio = 0.7, learn_rate = 0.01,hidden_dim = 18, n_layers = 1, device = device, \
#                       batch_size = 1000,drop_prob = 0.2, EPOCHS = 1000, output_dim = 2, method = "RNN", \
#                       out_var = "was_preterm",label_smooth= True, multi_outcome= True):
    
#     ##check argument validity
#     if out_var != "was_preterm" and out_var != "was_early_preterm":
#         raise ValueError("outcome must be was_preterm or was_early_preterm")
    
#     ##obtain data by goal
#     if out_var == "was_preterm":
#         data = mydata_preterm
#     else:
#         data = mydata_epreterm
        
#     project_list = data['project'].unique()
#      ##accuracy, sen, spe, auc score
#     train_loss = {}
#     val_loss = {}
#     acc = {}
#     sen = {}
#     spe = {}
#     auc = {}
#     precision = {}
#     result_tabs = {}
    
    
#     for project in project_list:
#         train_list = data[data['project']!= project].groupby('participant_id').first().sort_index(axis = 0)
#         train_id = list(train_list.groupby(out_var).sample(frac = train_ratio, random_state = 100).sort_index(axis = 0).index)
#         val_id = np.setdiff1d(list(train_list.index),train_id)
#         test_id = list(data[data['project']== project].groupby('participant_id').first().sort_index(axis = 0).index)
#         train_x, train_y = tensor_generator(data,train_id,features = features, \
#                                             out_var= "was_preterm", \
#                                             label_smooth= label_smooth, multi_outcome= multi_outcome)
#         val_x, val_y = tensor_generator(data,val_id,features = features, \
#                                             out_var= "was_preterm", \
#                                             label_smooth= label_smooth, multi_outcome= multi_outcome)
        
#         train_dataloader = dataloader(train_x, train_y,batch_size =  batch_size)
#         val_dataloader = dataloader(val_x, val_y,batch_size =  batch_size)
#         model, train_loss[project],val_loss[project] = train_epoch(train_dataloader,val_dataloader, learn_rate = learn_rate, \
#                 hidden_dim = hidden_dim, n_layers = n_layers, drop_prob = drop_prob, \
#                 device = device, EPOCHS = EPOCHS, output_dim = output_dim, method = method)
        
#         acc[project],sen[project],spe[project],auc[project],precision[project],result_tabs[project] = \
#         test_metrics(model, data,test_id,features, out_var = out_var, multi_outcome = multi_outcome)
    
#     return train_loss,val_loss, result_tabs ,pd.DataFrame([acc,sen,spe,precision,auc], \
#                                                  index = ['accuracy','sensitivity','specificity','precision','AUC'])
    
   
    